In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import os

In [2]:
!pip3 install kagglehub --quiet

In [3]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database")
print("Path to dataset files:", path)

Path to dataset files: /Users/dakshbhatnagar/.cache/kagglehub/datasets/terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database/versions/1


In [4]:
FILE_PATH = os.listdir(path)[0]
SQL_FILE_PATH = os.path.join(path,FILE_PATH )
#Setting Up the connection
conn = sqlite3.connect(SQL_FILE_PATH)

![img](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F2473556%2F23a7d4d8cd99e36e32e57303eb804fff%2Fdb-schema.png?generation=1714391550829633&alt=media)

We will be answering 20 questions using SQL from this database to further our understanding of the business and make data driven decisions.


1. **Find the total number of orders and average order price for each seller_city.**

In [5]:
query = '''

SELECT t2.seller_city,
        COUNT(t2.seller_city) AS count,
        ROUND(AVG(COALESCE(t1.price, 0) + COALESCE(t1.freight_value, 0)), 2) AS avg_order_price
FROM sellers t2 LEFT JOIN order_items t1 ON t1.seller_id = t2.seller_id
GROUP BY t2.seller_city
order by count desc
'''
seller_city = pd.read_sql(query, conn)
seller_city.head()

,seller_city,count,avg_order_price
0,sao paulo,27983,113.70
1,ibitinga,7750,98.36
2,curitiba,3016,179.67
3,santo andre,2964,92.72
4,belo horizonte,2593,98.04


2. **List the top 5 product categories by revenue with their English translations, ordered by total revenue**

In [6]:
query = '''
with cte as (
        select t3.product_category_name,
             (count(t3.product_category_name) * (t1.price + t1.freight_value)) as Revenue
        from order_items t1 left join orders t2 on t1.order_id = t2.order_id
                            left join products t3 on t1.product_id =  t3.product_id
        where t2.order_status = 'delivered'
        group by t3.product_category_name
        order by Revenue desc
        limit 5)

select t5.product_category_name_english,
       t4.Revenue
from cte t4 join product_category_name_translation t5
on t4.product_category_name = t5.product_category_name
'''
top5_product_category = pd.read_sql(query, conn)
top5_product_category

,product_category_name_english,Revenue
0,construction_tools_construction,1260828.20
1,watches_gifts,1128912.12
2,health_beauty,1125009.90
3,computers_accessories,872409.72
4,sports_leisure,806593.77


3. **Rank sellers by their total revenue within each state and show their relative performance percentile.**

In [7]:
query = '''
with cte as (SELECT t2.seller_id,
                    t2.seller_state,
                    ROUND(SUM(COALESCE(t1.price, 0) + COALESCE(t1.freight_value, 0)), 2) AS total_revenue
                    FROM sellers t2
                    LEFT JOIN order_items t1 ON t1.seller_id = t2.seller_id
                    GROUP BY t2.seller_id, t2.seller_state),

rank_cte as (select *,
            rank() over(partition by seller_state order by total_revenue desc) as state_rank,
            count(*) over(partition by seller_state) as total_sellers
            from cte),

percentile_cte as (select *,
                    COALESCE(ROUND((1 - (state_rank - 1)*1.0 /
                    COALESCE(total_sellers - 1, 0)) * 100,2),0) as percentile
            from rank_cte)

select seller_id,
        seller_state,
        total_revenue,
        state_rank, total_sellers,
        percentile || '%' as performance_percentile,
        CASE WHEN percentile >= 90 THEN 'Top 10%'
             WHEN percentile >= 75 THEN 'Top 25%'
             WHEN percentile >= 50 THEN 'Top 50%'
             ELSE 'Bottom 50%'
        END as performance_tier
from percentile_cte
ORDER BY seller_state, state_rank
'''
seller_ranking = pd.read_sql(query, conn)
seller_ranking.head()

,seller_id,seller_state,total_revenue,state_rank,total_sellers,performance_percentile,performance_tier
0,4be2e7f96b4fd749d52dff41f80e39dd,AC,299.84,1,1,0%,Bottom 50%
1,327b89b872c14d1c0be7235ef4871685,AM,1258.80,1,1,0%,Bottom 50%
2,53243585a1d6dc2643021fd1853d8905,BA,235856.68,1,19,100.0%,Top 10%
3,c72de06d72748d1a0dfb2125be43ba63,BA,18733.18,2,19,94.44%,Top 10%
4,75d34ebb1bd0bd7dde40dd507b8169c3,BA,16479.51,3,19,88.89%,Top 25%


4. **Find the most profitable product categories in each state, considering the seller and customer locations.**

In [8]:
query = '''
with cte as (select t3.product_category_name,
        t4.seller_state,
         (count(t3.product_category_name) * (t1.price + t1.freight_value)) as Revenue
from order_items t1 left join orders t2 on t1.order_id = t2.order_id
                    left join products t3 on t1.product_id =  t3.product_id
                    left join sellers t4 on t1.seller_id = t4.seller_id
group by t3.product_category_name, t4.seller_state),

interim as (select *,
                row_number() over(partition by seller_state order by Revenue desc ) as rn
                from cte)

select t5.seller_state,
        t6.product_category_name_english, t5.Revenue from interim t5
join product_category_name_translation t6
on t5.product_category_name = t6.product_category_name
where t5.rn = 1
'''
most_profitable_categories = pd.read_sql(query, conn)
most_profitable_categories.sample(5, random_state=42)

,seller_state,product_category_name_english,Revenue
15,MT,perfumery,9259.80
9,SE,computers_accessories,842.72
0,CE,health_beauty,18071.40
8,RS,computers_accessories,62700.00
17,PB,watches_gifts,12238.72


5. **Analyze the distribution of payment_types and payment_installments across different order values.**

In [9]:
query = '''
SELECT
    payment_type,
    payment_installments,
    COUNT(*) AS order_count,
    SUM(payment_value) AS total_payment_value,
    ROUND(AVG(payment_value), 2) AS avg_payment_value
FROM order_payments
GROUP BY payment_type, payment_installments
ORDER BY payment_type, payment_installments;
'''
payment_type_analysis = pd.read_sql(query, conn)
payment_type_analysis.sample(5, random_state=42)

,payment_type,payment_installments,order_count,total_payment_value,avg_payment_value
9,credit_card,8,4268,1313423.34,307.74
25,debit_card,1,1529,217989.79,142.57
8,credit_card,7,1626,305157.39,187.67
21,credit_card,21,3,731.10,243.70
0,boleto,1,19784,2869361.27,145.03


6. **Find the customers who have placed orders but have not made any reviews.**

In [10]:
query = '''
select t1.customer_id,
      count(t2.review_id) as review_count
from orders t1 left join order_reviews t2 on t1.order_id = t2.order_id
group by t1.customer_id
having count(t2.review_id) = 0
'''
review_count = pd.read_sql(query, conn)
review_count.sample(5, random_state=42)

,customer_id,review_count
668,e0d3183e8ff8cac00ec77df643595074,0
324,690ba4d5f5c3850bc7377fe13e6f353f,0
624,d0cc28a5ac316de296f6cb3babc9e665,0
690,e6e34332de50d27b69260b393f05eb43,0
473,9c3a67b63043d1626a51a23c256e0caa,0


7. **For each business_segment, calculate the conversion rate and average time to close (won_date - first_contact_date)**

In [11]:
query = '''
SELECT
    COALESCE(t2.business_segment, 'Unknown') AS business_segment,
    ROUND(SUM(CASE WHEN t2.won_date IS NOT NULL THEN 1 ELSE 0 END) * 100.0 / COUNT(*),2) AS conversion_rate,
    ROUND(AVG(CASE WHEN t2.won_date IS NOT NULL THEN julianday(t2.won_date) - julianday(t1.first_contact_date) END),2)
                        AS avg_time_to_close
FROM leads_qualified t1
LEFT JOIN leads_closed t2
ON t1.mql_id = t2.mql_id
GROUP BY COALESCE(t2.business_segment, 'Unknown');
'''
biz_seg_conv = pd.read_sql(query, conn)
biz_seg_conv.sample(5, random_state=42)

,business_segment,conversion_rate,avg_time_to_close
15,handcrafted,100.0,61.12
19,home_office_furniture,100.0,22.41
27,phone_mobile,100.0,30.16
26,pet,100.0,29.86
8,computers,100.0,55.68


8. **Identify customer_states where the average order value is significantly higher than the overall average.**

In [12]:
query = '''
WITH state_avg AS (
    SELECT
        t1.customer_state,
        AVG(t3.price + t3.freight_value) AS avg_order_value
    FROM customers t1
    LEFT JOIN orders t2 ON t1.customer_id = t2.customer_id
    LEFT JOIN order_items t3 ON t2.order_id = t3.order_id
    GROUP BY t1.customer_state
),
overall_avg AS (
    SELECT AVG(avg_order_value) AS overall_avg_order_value
    FROM state_avg),

final_cte as (SELECT
    state_avg.customer_state,
    ROUND(state_avg.avg_order_value,2) as avg_order_value,
    ROUND((state_avg.avg_order_value - overall_avg.overall_avg_order_value) /
                overall_avg.overall_avg_order_value * 100.0, 2) AS percentage_above_overall
FROM state_avg, overall_avg)

select * from final_cte where percentage_above_overall > 5
'''
higher_order_value = pd.read_sql(query, conn)
higher_order_value

,customer_state,avg_order_value,percentage_above_overall
0,AC,213.80,21.73
1,AL,216.73,23.40
2,AP,198.33,12.92
3,CE,186.47,6.17
4,PA,201.53,14.74
5,PB,234.20,33.35
6,PI,199.51,13.59
7,RN,192.62,9.67
8,RO,207.04,17.88
9,RR,193.55,10.20


9. **Compare seller performance based on average delivery time and review scores within the same city.**

In [13]:
query = '''
WITH seller_performance AS (
    SELECT
        t2.seller_id,
        t3.seller_city,
        ROUND(AVG(CASE
            WHEN t1.order_approved_at IS NOT NULL
            THEN julianday(t1.order_delivered_customer_date) - julianday(t1.order_approved_at)
            END), 2) AS avg_delivery_time,
        ROUND(AVG(t4.review_score), 2) AS avg_review_score
    FROM orders t1
    LEFT JOIN order_items t2 ON t1.order_id = t2.order_id
    LEFT JOIN sellers t3 ON t2.seller_id = t3.seller_id
    LEFT JOIN order_reviews t4 ON t1.order_id = t4.order_id
    WHERE t3.seller_id IS NOT NULL
    GROUP BY t2.seller_id, t3.seller_city
),
ranked_performance AS (
    SELECT
        seller_id,
        seller_city,
        avg_delivery_time,
        avg_review_score,
        RANK() OVER (PARTITION BY seller_city ORDER BY avg_delivery_time ASC) AS delivery_rank,
        RANK() OVER (PARTITION BY seller_city ORDER BY avg_review_score DESC) AS review_rank
    FROM seller_performance
)
SELECT
    seller_id,
    seller_city,
    avg_delivery_time,
    avg_review_score,
    delivery_rank,
    review_rank
FROM ranked_performance
ORDER BY seller_city, delivery_rank, review_rank;
'''
seller_performance_within_city = pd.read_sql(query, conn)
seller_performance_within_city.sample(5, random_state=42)

,seller_id,seller_city,avg_delivery_time,avg_review_score,delivery_rank,review_rank
1947,6288c69c4ce638e59925e59193f98b16,santo andre,9.38,3.95,22,30
1309,2a7c6c0b0d5efde2bf75ec1e1cec2d41,maringa,9.86,3.33,15,37
1606,5e9f5bd34e0830569f05c9d4f56e04a8,porto alegre,10.02,4.50,10,8
2626,6d66611d7c44cc30ce351abc49a68421,sao paulo,11.67,3.91,470,445
2440,876182d1918783bbe15233febfbd4926,sao paulo,8.48,4.00,284,381


10. **Analyze monthly order patterns for each product_category_name to identify seasonal trends.**

In [14]:
query = '''
WITH cte AS (
    SELECT
        t3.product_category_name,
        strftime("%Y-%m", t2.order_purchase_timestamp) AS month,
        COUNT(*) AS order_count
    FROM order_items t1
    LEFT JOIN orders t2 ON t1.order_id = t2.order_id
    LEFT JOIN products t3 ON t1.product_id = t3.product_id
    WHERE t2.order_purchase_timestamp IS NOT NULL
    GROUP BY t3.product_category_name,
        month)

SELECT
    t5.product_category_name_english as product_category,
    t4.month,
    t4.order_count
FROM cte t4
join product_category_name_translation t5
on t4.product_category_name = t5.product_category_name
WHERE t5.product_category_name IS NOT NULL
ORDER BY
    t4.month,
    t5.product_category_name_english
'''
trends = pd.read_sql(query, conn)
trends.head()

,product_category,month,order_count
0,furniture_decor,2016-09,2
1,health_beauty,2016-09,3
2,telephony,2016-09,1
3,air_conditioning,2016-10,10
4,audio,2016-10,2


11. **Calculate the average review score for each product category.**

In [15]:
query = '''
WITH cte AS (
    SELECT
        t4.product_category_name,
        ROUND(AVG(t2.review_score), 2) AS avg_review_score
    FROM orders t1
    LEFT JOIN order_reviews t2 ON t1.order_id = t2.order_id
    LEFT JOIN order_items t3 ON t2.order_id = t3.order_id
    LEFT JOIN products t4 ON t3.product_id = t4.product_id
    where t4.product_category_name is not null
    GROUP BY t4.product_category_name)

SELECT
    t6.product_category_name_english,
    t5.avg_review_score
FROM cte t5
LEFT JOIN product_category_name_translation t6 ON t5.product_category_name = t6.product_category_name
order by t5.avg_review_score desc
'''
avg_review_score = pd.read_sql(query, conn)
avg_review_score.head()

,product_category_name_english,avg_review_score
0,cds_dvds_musicals,4.64
1,fashion_childrens_clothes,4.50
2,books_general_interest,4.45
3,costruction_tools_tools,4.44
4,flowers,4.42


12. **Identify the products that have been reviewed by more than 5 customers.**

In [16]:
query = '''
select product_id,
        count(review_id) as review_count
from order_reviews t1 join order_items t2 on t1.order_id = t2.order_id and t1.review_id is not null
group by product_id
having count(review_id) > 5
order by review_count desc

'''
product_review_count = pd.read_sql(query, conn)
product_review_count.head()

,product_id,review_count
0,aca2eb7d00ea1a7b8ebd4e68314663af,524
1,422879e10f46682990de24d770e7f83d,486
2,99a4788cb24856965c36a24e339b6058,482
3,389d119b48cf3043d311335e499d9c6b,391
4,368c6c730842d78016ad823897a372db,388


13. **Find the customers who have not placed any orders after '2018-01-01'.**

In [17]:
query = '''
select * from customers
where customer_id not in (select distinct customer_id
        from orders
        where strftime("%Y-%m-%d", order_purchase_timestamp) > '2018-01-01')
'''
no_order_after_2018 = pd.read_sql(query, conn)
no_order_after_2018.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
2,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
3,9fb35e4ed6f0a14a4977cd9aea4042bb,2a7745e1ed516b289ed9b29c7d0539a5,39400,montes claros,MG
4,b2d1536598b73a9abd18e0d75d92f0a3,918dc87cd72cd9f6ed4bd442ed785235,18682,lencois paulista,SP


14. **Display the top 5 products by sales quantity.**

In [18]:
query = '''
select t2.product_id,
        count(t2.order_id) as qty_sold
from orders t1 left join order_items t2 on t1.order_id = t2.order_id
group by t2.product_id
order by 2 desc
limit 5
'''
top5_products = pd.read_sql(query, conn)
top5_products

,product_id,qty_sold
0,aca2eb7d00ea1a7b8ebd4e68314663af,527
1,99a4788cb24856965c36a24e339b6058,488
2,422879e10f46682990de24d770e7f83d,484
3,389d119b48cf3043d311335e499d9c6b,392
4,368c6c730842d78016ad823897a372db,388


15. **Determine the average order processing time for orders placed on weekends.**

In [19]:
query = '''
SELECT round(avg(julianday(order_delivered_customer_date) -
                julianday(order_purchase_timestamp)),2) as avg_order_processing_time
FROM orders
WHERE strftime('%w', order_purchase_timestamp) IN ('0', '6')
'''
avg_order_processing_time = pd.read_sql(query, conn).values.flatten()[0]
print(f"The avg order processing time for order placed on weekends is {avg_order_processing_time} days")

The avg order processing time for order placed on weekends is 12.63 days


16. **Calculate the average freight_value to price ratio for each product_category_name by state.**

In [20]:
query = '''
with cte as (SELECT
    t2.seller_state,
    t4.product_category_name,
    ROUND(AVG(t1.freight_value / t1.price), 2) AS freight_to_price_ratio
FROM order_items t1
JOIN sellers t2 ON t1.seller_id = t2.seller_id
JOIN  products t4 ON t1.product_id = t4.product_id
GROUP BY t2.seller_state, t4.product_category_name )


select t5.seller_state,
        t6.product_category_name_english,
        t5.freight_to_price_ratio

from cte t5
join product_category_name_translation t6
on t5.product_category_name = t6.product_category_name
ORDER BY t5.freight_to_price_ratio DESC

'''
freight_to_price_ratio = pd.read_sql(query,conn)
freight_to_price_ratio.head()

,seller_state,product_category_name_english,freight_to_price_ratio
0,PR,food_drink,1.81
1,BA,furniture_decor,1.77
2,PR,signaling_and_security,1.68
3,PE,health_beauty,1.55
4,PE,housewares,1.43


17. **Find which customer_states have the highest percentage of multi-installment payments**

In [21]:
query = '''
SELECT
    t3.customer_state,
    ROUND(
        SUM(CASE WHEN t1.payment_installments > 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS installment_pct
FROM order_payments t1
JOIN orders t2 ON t1.order_id = t2.order_id
JOIN customers t3 ON t2.customer_id = t3.customer_id
GROUP BY t3.customer_state
ORDER BY installment_pct DESC
LIMIT 1;
'''
installment_percent = pd.read_sql(query, conn)
installment_percent

,customer_state,installment_pct
0,AL,64.4


18. **Calculate the average time between order delivery and review creation by product category.**

In [22]:
query = '''
with cte as (
select t4.product_category_name,
        round(avg(julianday(strftime("%Y-%m-%d", t2.review_creation_date)) -
                  julianday(strftime("%Y-%m-%d", t1.order_delivered_customer_date) )),2) as avg_time_in_days
from orders t1 join order_reviews t2 on t1.order_id = t2.order_id
join order_items t3 on t2.order_id = t3.order_id
join products t4 on t3.product_id = t4.product_id
group by t4.product_category_name
order by 2 desc)

select
        t6.product_category_name_english,
        t5.avg_time_in_days || " Days" as avg_time

from cte t5
join product_category_name_translation t6
on t5.product_category_name = t6.product_category_name
ORDER BY t5.avg_time_in_days DESC

'''
avg_time_delivery_review = pd.read_sql(query, conn)
avg_time_delivery_review.head()

,product_category_name_english,avg_time
0,furniture_bedroom,1.19 Days
1,cine_photo,1.17 Days
2,la_cuisine,1.0 Days
3,cds_dvds_musicals,1.0 Days
4,fashion_childrens_clothes,1.0 Days


19. **Analyze which business_segments in leads_closed have the highest declared_monthly_revenue.**

In [23]:
query = '''
with cte as (select t1.business_segment,
                sum(declared_monthly_revenue) as revenue
        from leads_closed t1
left join sellers t2 on t1.seller_id =  t2.seller_id
group by t1.business_segment),

interim as (select *, row_number() over(partition by business_segment
            order by revenue desc) as rn from cte)

select business_segment,
        (CAST(revenue/1000 as integer)  || "k") as total_revenue
from interim where rn = 1 and revenue!=0
order by CAST(revenue/1000 as integer) desc
'''
revenue_state_segment = pd.read_sql(query, conn)
revenue_state_segment

,business_segment,total_revenue
0,construction_tools_house_garden,50695k
1,phone_mobile,8000k
2,home_decor,710k
3,other,626k
4,pet,300k
5,health_beauty,295k
6,audio_video_electronics,270k
7,toys,259k
8,car_accessories,185k
9,household_utilities,163k


20. **Compare conversion rates between different origins in leads_qualified across business segments**

In [24]:
query = '''
select t2.business_segment,
    round(sum(case when t2.won_date is not null and t1.origin = 'social'
    then 1 else 0 end)*100.0/count(*),2) || "%" as social,
    round(sum(case when t2.won_date is not null and t1.origin = 'paid_search'
    then 1 else 0 end)*100.0/count(*),2) || "%" as paid_search,
    round(sum(case when t2.won_date is not null and t1.origin = 'organic_search'
    then 1 else 0 end)*100.0/count(*),2) || "%" as organic_search,
    round(sum(case when t2.won_date is not null and t1.origin = 'email'
    then 1 else 0 end)*100.0/count(*),2) || "%" as email,
    round(sum(case when t2.won_date is not null and t1.origin = 'unknown'
    then 1 else 0 end)*100.0/count(*),2) || "%" as unknown,
    round(sum(case when t2.won_date is not null and t1.origin = 'referral'
    then 1 else 0 end)*100.0/count(*),2) || "%" as referral,
    round(sum(case when t2.won_date is not null and t1.origin = 'direct traffic'
    then 1 else 0 end)*100.0/count(*),2) || "%" as direct_traffic,
    round(sum(case when t2.won_date is not null and t1.origin = 'display'
    then 1 else 0 end)*100.0/count(*),2) || "%" as social,
    round(sum(case when t2.won_date is not null and t1.origin = 'other_publicities'
    then 1 else 0 end)*100.0/count(*),2) || "%" as other_publicities,
    round(sum(case when t2.won_date is not null and t1.origin = 'other'
    then 1 else 0 end)*100.0/count(*),2) || "%" as other

from leads_qualified t1 left join leads_closed t2 on t1.mql_id = t2.mql_id
where t2.business_segment is not null
group by t2.business_segment

'''
business_origin_conv_rates = pd.read_sql(query, conn)
business_origin_conv_rates

,business_segment,social,paid_search,organic_search,email,unknown,referral,direct_traffic,social,other_publicities,other
0,air_conditioning,0.0%,0.0%,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%
1,audio_video_electronics,14.06%,23.44%,32.81%,1.56%,17.19%,1.56%,0.0%,0.0%,0.0%,0.0%
2,baby,20.0%,20.0%,10.0%,0.0%,30.0%,10.0%,0.0%,0.0%,0.0%,0.0%
3,bags_backpacks,9.09%,18.18%,22.73%,0.0%,27.27%,0.0%,0.0%,0.0%,0.0%,0.0%
4,bed_bath_table,4.55%,31.82%,27.27%,0.0%,27.27%,0.0%,0.0%,0.0%,0.0%,4.55%
5,books,0.0%,22.22%,55.56%,0.0%,11.11%,0.0%,0.0%,0.0%,0.0%,0.0%
6,car_accessories,9.09%,25.97%,33.77%,1.3%,18.18%,2.6%,0.0%,1.3%,0.0%,0.0%
7,computers,2.94%,32.35%,23.53%,0.0%,26.47%,5.88%,0.0%,2.94%,0.0%,0.0%
8,construction_tools_house_garden,7.25%,20.29%,27.54%,2.9%,27.54%,2.9%,0.0%,2.9%,0.0%,0.0%
9,fashion_accessories,10.53%,15.79%,36.84%,5.26%,26.32%,0.0%,0.0%,0.0%,0.0%,0.0%


In [25]:
#Closing the connection
conn.close()